# Google Earth Engine Python API

[Google Earth Engine](https://developers.google.com/earth-engine/) \
\
Google Earth Engine is a cloud based platform which provides a geospatial processing service. The purpose of Earth Engine is to:
* Provide an interactive platform for geospatial algorithm development at scale
* Enable high-impact, data-driven science
* Make substantive progress on global challenges that involve large geospatial datasets

# Steps Included in Assignment A3

### All the assignment (included this) is done using **Terminal** and **Jupyter Lab**

* Move to the working directory

* Creating directory ./A3/ using: \
  ```mkdir A3```

* Creating a new environment **earthengine**: \
  ```conda create -n earthengine```

* Activate the environment **earthengine**: \
  ```conda activate earthengine```
  
* Install the earthengine package. Earth Engine Python API is distributed as a *conda-forge package*. \
   ```conda install -c conda-forge earthengine-api```

* Installing ipykernel to properly manage the environment. \
    ```conda install ipykernel``` \
    ```python ipykernel install --user --name earthengine --display-name "Earth Engine"```
  
* Opening Jupyter Lab: ensure Jupyter is already installed in the environment or base:
    ```jupyter lab```
  
* Once the Lab is opened, go to A3 and create a new ipython notebook file:

In [1]:
#importing earthengine and folium
import ee
import folium

In [2]:
ee.Authenticate()
ee.Initialize()

## Objective

* Accesing Earth Engine Data catalog
* Selecting Landsat 8 Image Collection
* Filtering the collection and selecting the images which are in user-required date and user-required location.
* I ignore handling of cloud cover for this assignment. 😜 \
**If anyone requires handling cloud cover, even for filtering cloud cover within in specified location; Drop me a private message.**
* Calculating mean of the image collection
* Creating indices: NDBI, NDVI, NDWI : *Lemme know if you want more indices to be used*. \
**I could go for performing TimeSeries analysis if I have enough time.**
* Displaying all these Indices 

In [3]:
ROI = {
    'cityName'  : 'Salzburg',
    'LatMax'    : 47.85,
    'LatMin'    : 47.75,
    'LonMax'    : 13.20,
    'LonMin'    : 12.90,
    'start_date': '2023-01-01',
    'end_date'  : '2023-03-20',
}

collection = 'LANDSAT/LC08/C02/T1_TOA'

In [4]:
out_ext = ee.Geometry.Rectangle(
    [
        ROI['LonMin'],
        ROI['LatMin'],
        ROI['LonMax'],
        ROI['LatMax']
    ]
)

In [5]:
image_collection = ee.ImageCollection(collection).filterBounds(out_ext).filterDate(ROI['start_date'],ROI['end_date'])
print(f'Total Image in the specified date and area of interest is: {image_collection.size().getInfo()}')
mean_image = image_collection.mean()
clipped_image = mean_image.clip(out_ext)
blue  = clipped_image.select('B2')
green = clipped_image.select('B3')
red   = clipped_image.select('B4')
nir   = clipped_image.select('B5')
swir  = clipped_image.select('B6')

Total Image in the specified date and area of interest is: 9


In [6]:
NDBI = swir.subtract(nir).divide(swir.add(nir))
NDVI = nir.subtract(red).divide(nir.add(red))
NDWI_leaf = nir.subtract(swir).divide(nir.add(swir))        ## Water content in leaf
NDWI_water = green.subtract(nir).divide(green.add(nir))     ## Water content in water bodies

In [7]:
ndvi_palette = ['blue', 'white', 'green']

ndvi_vis_params = {
    'min': -0.3,
    'max': 0.3,
    'palette': ndvi_palette
}

# Get the Map ID for NDVI
ndvi_mapid = NDVI.getMapId(ndvi_vis_params)
out_ext_center = out_ext.centroid()
center_lat = out_ext_center.getInfo()['coordinates'][1]
center_lon = out_ext_center.getInfo()['coordinates'][0]

# Create a folium map
map_ndvi = folium.Map(location=[center_lat, center_lon], zoom_start=13, tiles='CartoDB positron')

# Add NDVI tile layer to the map
folium.TileLayer(
    tiles=ndvi_mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='NDVI',
).add_to(map_ndvi)

# Add layer control
map_ndvi.add_child(folium.LayerControl())

# Display the map
map_ndvi

In [8]:
ndwi_palette = ['0000ff', '00ffff', 'ffff00', 'ff0000', 'ffffff']

ndwi_vis_params = {
    'min': -0.2,
    'max': 0.1,
    'palette': ndwi_palette
}

# Get the Map ID for NDWI
ndwi_mapid = NDWI_water.getMapId(ndwi_vis_params)
out_ext_center = out_ext.centroid()
center_lat = out_ext_center.getInfo()['coordinates'][1]
center_lon = out_ext_center.getInfo()['coordinates'][0]

# Create a folium map
map_ndwi = folium.Map(location=[center_lat, center_lon], zoom_start=13, tiles='CartoDB positron')

# Add NDWI tile layer to the map
folium.TileLayer(
    tiles=ndwi_mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='NDWI',
).add_to(map_ndwi)

# Add layer control
map_ndwi.add_child(folium.LayerControl())

# Display the map
map_ndwi

In [9]:
ndwi_leaf_palette = ['0000ff', '00ffff', 'ffff00', 'ff0000', 'ffffff']

ndwi_leaf_vis_params = {
    'min': 0,
    'max': 1,
    'palette': ndwi_leaf_palette
}

# Get the Map ID for NDWI_leaf
ndwi_leaf_mapid = NDWI_leaf.getMapId(ndwi_leaf_vis_params)
out_ext_center = out_ext.centroid()
center_lat = out_ext_center.getInfo()['coordinates'][1]
center_lon = out_ext_center.getInfo()['coordinates'][0]

# Create a folium map
map_ndwi_leaf = folium.Map(location=[center_lat, center_lon], zoom_start=13, tiles='CartoDB positron')

# Add NDWI_leaf tile layer to the map
folium.TileLayer(
    tiles=ndwi_leaf_mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='NDWI_leaf',
).add_to(map_ndwi_leaf)

# Add layer control
map_ndwi_leaf.add_child(folium.LayerControl())

# Display the map
map_ndwi_leaf

In [10]:
ndbi_palette = ['white', 'red']

ndbi_vis_params = {
    'min': -0.6,
    'max': 0.4,
    'palette': ndbi_palette
}

# Get the Map ID for NDBI
ndbi_mapid = NDBI.getMapId(ndbi_vis_params)
out_ext_center = out_ext.centroid()
center_lat = out_ext_center.getInfo()['coordinates'][1]
center_lon = out_ext_center.getInfo()['coordinates'][0]

# Create a folium map
map_ndbi = folium.Map(location=[center_lat, center_lon], zoom_start=13, tiles='CartoDB positron')

# Add NDBI tile layer to the map
folium.TileLayer(
    tiles=ndbi_mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='NDBI',
).add_to(map_ndbi)

# Add layer control
map_ndbi.add_child(folium.LayerControl())

# Display the map
map_ndbi